In [43]:
import numpy as np
import optuna

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [44]:
RAND_STATE = 42

In [45]:
# Mock dataset for demonstration
class TextDataset(Dataset):
    # def __init__(self, vocabulary_size, sequence_length, num_samples):
    #     self.data = torch.randint(0, vocabulary_size, (num_samples, sequence_length))
    #     self.labels = torch.randint(0, 2, (num_samples,))
    def __init__(self, texts, labels, sequence_length):
        self.texts = texts
        self.labels = labels
        self.sequence_length = sequence_length
        self.vocab = self.build_vocab(texts)
        self.encoded_texts = [self.encode_text(text) for text in texts]

    def build_vocab(self, texts):
        unique_words = set(word for text in texts for word in text.lower().split())
        vocab = {word: i + 1 for i, word in enumerate(unique_words)}  # +1 for padding token at index 0
        return vocab
    
    def encode_text(self, text):
        return [self.vocab.get(word, 0) for word in text.lower().split()][:self.sequence_length] + [0] * (self.sequence_length - len(text.split()))

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return torch.tensor(self.encoded_texts[idx]), torch.tensor(self.labels[idx])
    

# Define the Q-network model
# class DQN(nn.Module):
#     def __init__(self, vocabulary_size, embedding_dim, hidden_dim, num_classes):
#         super(DQN, self).__init__()
#         self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, num_classes)

#     def forward(self, x):
#         embeds = self.embedding(x)
#         lstm_out, _ = self.lstm(embeds)
#         q_values = self.fc(lstm_out[:, -1])
#         return q_values

# class DQN(nn.Module):
#     def __init__(self, vocabulary_size, embedding_dim, hidden_dim, num_classes):
#         super(DQN, self).__init__()
#         self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, num_classes)

#     def forward(self, x):
#         embeds = self.embedding(x)
#         lstm_out, _ = self.lstm(embeds)
#         q_values = self.fc(lstm_out[:, -1])
#         return q_values
    
class DQN(nn.Module):
    def __init__(self, vocabulary_size, embedding_dim, hidden_dim, num_classes, dropout_rate=0.5, pre_trained_embeddings=None):
        super(DQN, self).__init__()
        self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
        if pre_trained_embeddings is not None:
            self.embedding.weight = nn.Parameter(pre_trained_embeddings)
            self.embedding.weight.requires_grad = False  # Or True if you want to fine-tune

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True, num_layers=2) # Stacked LSTMs
        self.dropout = nn.Dropout(dropout_rate)  # Use dropout_rate from arguments
        self.fc = nn.Linear(hidden_dim * 2, num_classes)  # Adjust for bidirectional LSTM
        self.relu = nn.ReLU()

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, _ = self.lstm(embeds)
        out = self.dropout(lstm_out[:, -1])
        out = self.relu(out)
        q_values = self.fc(out)
        return q_values

In [46]:
def train(model, device, train_loader, optimizer, epoch, log_interval=10):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        # Using CrossEntropyLoss which combines LogSoftmax and NLLLoss
        loss = nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

def validate(model, device, validation_loader):
    model.eval()
    validation_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in validation_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            validation_loss += nn.CrossEntropyLoss()(output, target).item()  # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    validation_loss /= len(validation_loader.dataset)
    validation_acc = correct / len(validation_loader.dataset)
    print(f'\nValidation set: Average loss: {validation_loss:.4f}, Accuracy: {correct}/{len(validation_loader.dataset)} ({100. * correct / len(validation_loader.dataset):.0f}%)\n')
    return validation_loss, validation_acc

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.CrossEntropyLoss()(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

In [47]:
# Example texts and labels
texts = [
    "Great movie loved it", "Did not like the movie", "Awesome plot and acting",
    "Boring and too long", "Wonderfully made film", "Not my cup of tea",
    "Thrilling and exciting from start to finish", "Could have been better", "A masterpiece", "Waste of time",
    "The director did a fantastic job", "Terrible performances", "The cinematography is gorgeous", 
    "I fell asleep halfway through", "An unforgettable journey", "I've seen better", 
    "A total letdown from a great director", "The soundtrack complemented the movie perfectly", 
    "Lacked depth in the storyline", "Characters were not developed well", "Laughed all the way through", 
    "Failed to deliver a compelling narrative", "Visual effects were stunning", "Plot was predictable", 
    "Highly recommend this movie", "Not worth your time", "An epic disappointment", 
    "Engaging from start to finish", "Script was beautifully written", "The movie felt rushed",
    "A visual masterpiece", "Forgettable experience", "Will watch it again", "Do not waste your money", 
    "The acting was subpar", "A story well told", "Lacks originality", "Captivating to the very end", 
    "Missed the mark on many levels", "One of the year's best films"
]
labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1]

# Splitting dataset into train+val and test
train_val_texts, test_texts, train_val_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=RAND_STATE)

# Splitting train+val into train and val
train_texts, validation_texts, train_labels, validation_labels = train_test_split(train_val_texts, train_val_labels, test_size=0.25, random_state=RAND_STATE)  # 0.25 x 0.8 = 0.2

# Creating datasets
sequence_length = 10  # Max number of words in a text
train_dataset = TextDataset(train_texts, train_labels, sequence_length)
validation_dataset = TextDataset(validation_texts, validation_labels, sequence_length)
test_dataset = TextDataset(test_texts, test_labels, sequence_length)

# Creating DataLoaders
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for data, label in train_loader:
    print(f"Train Encoded text: {data}")
    print(f"Train Label: {label}")
    break  # Just show one batch for brevity

for data, label in validation_loader:
    print(f"Validation Encoded text: {data}")
    print(f"Validation Label: {label}")
    break  # Just show one batch for brevity

for data, label in test_loader:
    print(f"Test Encoded text: {data}")
    print(f"Test Label: {label}")
    break  # Just show one batch for brevity

Train Encoded text: tensor([[28, 48,  0,  0,  0,  0,  0,  0,  0,  0],
        [51, 73,  0,  0,  0,  0,  0,  0,  0,  0],
        [25, 35, 11,  0,  0,  0,  0,  0,  0,  0],
        [49, 36, 66, 11,  0,  0,  0,  0,  0,  0]])
Train Label: tensor([1, 0, 0, 0])
Validation Encoded text: tensor([[ 7, 16, 26,  0,  0,  0,  0,  0,  0,  0],
        [11, 22,  1, 30,  0,  0,  0,  0,  0,  0],
        [11,  8,  3, 13,  5, 15,  0,  0,  0,  0],
        [18,  9, 17,  6,  0,  0,  0,  0,  0,  0]])
Validation Label: tensor([1, 0, 1, 1])
Test Encoded text: tensor([[28, 30, 25, 19,  4,  0,  0,  0,  0,  0],
        [12, 10, 23,  9, 12,  1,  8,  0,  0,  0],
        [ 7, 27, 22,  0,  0,  0,  0,  0,  0,  0],
        [ 6,  5, 15,  0,  0,  0,  0,  0,  0,  0]])
Test Label: tensor([0, 0, 0, 0])


In [48]:
# Parameters and Hyperparameters
vocabulary_size = 10000  # to adjust 
sequence_length = 50  # to adjust 
embedding_dim = 128
hidden_dim = 64
num_classes = 2
batch_size = 64
epochs = 10
learning_rate = 0.001

# Model, optimizer, and device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DQN(vocabulary_size, embedding_dim, hidden_dim, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    validate(model, device, validation_loader)

# After training, evaluate on the test set
test(model, device, test_loader)

Train Epoch: 1 [0/24 (0%)]	Loss: 0.736371

Validation set: Average loss: 0.1958, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.696769

Validation set: Average loss: 0.2099, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.612735

Validation set: Average loss: 0.2336, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.650687

Validation set: Average loss: 0.2460, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.604238

Validation set: Average loss: 0.2520, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.408486

Validation set: Average loss: 0.2528, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.569398

Validation set: Average loss: 0.2473, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.696696

Validation set: Average loss: 0.2390, Accuracy: 1/8 (12%)

Train Epoch: 9 [0/24 (0%)]	Loss: 0.492158

Validation set: Average loss: 0.2425, Accuracy: 1/8 (12%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.544051

Validation set: Average loss: 0.2315, 

In [49]:
# Parameters and Hyperparameters
num_classes = 2

def objective(trial):
    # Define the search space
    vocabulary_size = trial.suggest_categorical('vocabulary_size', [5000, 10000, 20000, 40000])
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    embedding_dim = trial.suggest_categorical('embedding_dim', [64, 128, 256])
    hidden_dim = trial.suggest_categorical('hidden_dim', [32, 64, 128])
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'RMSprop', 'SGD'])
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
    step_size = trial.suggest_int('step_size', 1, 100)
    gamma = trial.suggest_float('gamma', 0.1, 1.0, log=True)
    sequence_length = trial.suggest_categorical('sequence_length', [50, 100, 200, 400])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Device configuration
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Model setup with trial suggestions
    model = DQN(vocabulary_size, embedding_dim, hidden_dim, num_classes, dropout_rate=dropout_rate).to(device)

    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=learning_rate)

    if optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_name == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
    elif optimizer_name == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

    # Training loop
    epochs = 10  # Reduced for faster optimization cycles
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        val_loss, val_accuracy = validate(model, device, validation_loader)
        scheduler.step()

    # Set custom attributes for the trial
    trial.set_user_attr("val_loss", val_loss)
    trial.set_user_attr("val_accuracy", val_accuracy)
    
    # print(f"Returning from validate: val_loss={val_loss}, val_accuracy={val_accuracy}")
    # return val_loss

    # Objective: maximize validation accuracy by minimizing its negative value
    return -val_accuracy  # Return the negative accuracy

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)  # Number of trials can be adjusted

print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial

# Retrieve the validation loss and accuracy from the best trial
best_val_loss = trial.user_attrs["val_loss"]
best_val_accuracy = trial.user_attrs["val_accuracy"]

print(f'Best Validation Loss: {best_val_loss}')
print(f'Best Validation Accuracy: {best_val_accuracy}')
print('Best Trial Parameters:')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

[I 2024-04-02 19:20:44,611] A new study created in memory with name: no-name-a3ba5b8e-f0ea-4624-b04e-68dccea7752a


Train Epoch: 1 [0/24 (0%)]	Loss: 0.706039

Validation set: Average loss: 0.0824, Accuracy: 7/8 (88%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.703627

Validation set: Average loss: 0.0826, Accuracy: 7/8 (88%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.723249

Validation set: Average loss: 0.0828, Accuracy: 7/8 (88%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.720792

Validation set: Average loss: 0.0830, Accuracy: 7/8 (88%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.709023

Validation set: Average loss: 0.0832, Accuracy: 7/8 (88%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.703518

Validation set: Average loss: 0.0834, Accuracy: 7/8 (88%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.712947

Validation set: Average loss: 0.0836, Accuracy: 7/8 (88%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.703514

Validation set: Average loss: 0.0838, Accuracy: 7/8 (88%)



[I 2024-04-02 19:20:44,797] Trial 0 finished with value: -0.875 and parameters: {'vocabulary_size': 20000, 'batch_size': 256, 'learning_rate': 0.005359418070276634, 'embedding_dim': 256, 'hidden_dim': 32, 'optimizer': 'SGD', 'dropout_rate': 0.42260298589055395, 'step_size': 97, 'gamma': 0.15067386263487642, 'sequence_length': 50}. Best is trial 0 with value: -0.875.


Train Epoch: 9 [0/24 (0%)]	Loss: 0.697011

Validation set: Average loss: 0.0840, Accuracy: 7/8 (88%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.712808

Validation set: Average loss: 0.0841, Accuracy: 7/8 (88%)

Train Epoch: 1 [0/24 (0%)]	Loss: 0.699766

Validation set: Average loss: 0.0873, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.691867

Validation set: Average loss: 0.0897, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.685220

Validation set: Average loss: 0.0919, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.677196

Validation set: Average loss: 0.0941, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.670826

Validation set: Average loss: 0.0960, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.665664

Validation set: Average loss: 0.0980, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.659947

Validation set: Average loss: 0.0998, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.656751

Validation set: Average loss: 0.1013, 

[I 2024-04-02 19:20:44,894] Trial 1 finished with value: -0.125 and parameters: {'vocabulary_size': 5000, 'batch_size': 256, 'learning_rate': 0.08010343202530555, 'embedding_dim': 256, 'hidden_dim': 32, 'optimizer': 'SGD', 'dropout_rate': 0.07570417638353161, 'step_size': 49, 'gamma': 0.22679175254478506, 'sequence_length': 50}. Best is trial 0 with value: -0.875.


...

Train Epoch: 8 [0/24 (0%)]	Loss: 0.670123

Validation set: Average loss: 0.0931, Accuracy: 1/8 (12%)

Train Epoch: 9 [0/24 (0%)]	Loss: 0.672974

Validation set: Average loss: 0.0932, Accuracy: 1/8 (12%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.672720

Validation set: Average loss: 0.0932, Accuracy: 1/8 (12%)

Train Epoch: 1 [0/24 (0%)]	Loss: 0.707741

Validation set: Average loss: 0.0846, Accuracy: 7/8 (88%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.705896

Validation set: Average loss: 0.0852, Accuracy: 7/8 (88%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.704694

Validation set: Average loss: 0.0857, Accuracy: 7/8 (88%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.705091

Validation set: Average loss: 0.0863, Accuracy: 7/8 (88%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.694877

Validation set: Average loss: 0.0868, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.693128

Validation set: Average loss: 0.0873, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.691218

Validation set: Average loss: 0.0878, 

[I 2024-04-02 19:20:51,184] Trial 27 finished with value: -0.125 and parameters: {'vocabulary_size': 10000, 'batch_size': 64, 'learning_rate': 0.016690511882538506, 'embedding_dim': 128, 'hidden_dim': 64, 'optimizer': 'SGD', 'dropout_rate': 0.48694598361314684, 'step_size': 74, 'gamma': 0.3177220898132646, 'sequence_length': 100}. Best is trial 0 with value: -0.875.
[I 2024-04-02 19:20:51,429] Trial 28 finished with value: -0.125 and parameters: {'vocabulary_size': 5000, 'batch_size': 128, 'learning_rate': 0.0006881957889045186, 'embedding_dim': 64, 'hidden_dim': 128, 'optimizer': 'RMSprop', 'dropout_rate': 0.3941369536723953, 'step_size': 93, 'gamma': 0.19664717937775247, 'sequence_length': 200}. Best is trial 0 with value: -0.875.


Train Epoch: 1 [0/24 (0%)]	Loss: 0.683285

Validation set: Average loss: 0.1410, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.629780

Validation set: Average loss: 0.1124, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.634779

Validation set: Average loss: 0.1598, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.672305

Validation set: Average loss: 0.0973, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.675825

Validation set: Average loss: 0.1379, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.619963

Validation set: Average loss: 0.1297, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.617677

Validation set: Average loss: 0.1155, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.551996

Validation set: Average loss: 0.1746, Accuracy: 1/8 (12%)

Train Epoch: 9 [0/24 (0%)]	Loss: 0.495986

Validation set: Average loss: 0.0536, Accuracy: 7/8 (88%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.916582

Validation set: Average loss: 0.1901, 

[I 2024-04-02 19:20:51,698] Trial 29 finished with value: -0.125 and parameters: {'vocabulary_size': 5000, 'batch_size': 256, 'learning_rate': 0.04604979392745883, 'embedding_dim': 256, 'hidden_dim': 32, 'optimizer': 'Adam', 'dropout_rate': 0.09772021676154796, 'step_size': 47, 'gamma': 0.13088423886711506, 'sequence_length': 50}. Best is trial 0 with value: -0.875.



Validation set: Average loss: 0.1608, Accuracy: 1/8 (12%)

Number of finished trials: 30
Best trial:
Best Validation Loss: 0.08412426710128784
Best Validation Accuracy: 0.875
Best Trial Parameters:
    vocabulary_size: 20000
    batch_size: 256
    learning_rate: 0.005359418070276634
    embedding_dim: 256
    hidden_dim: 32
    optimizer: SGD
    dropout_rate: 0.42260298589055395
    step_size: 97
    gamma: 0.15067386263487642
    sequence_length: 50
